In [ ]:



from core.pandas_utils import *
from core.caching_utils import cache_result
from transform.raw_results.config import *
from transform.processed_tss.ProcessedTimeSeries import TeslaProcessedTimeSeries



This notebook is used to compute the number of cycle for a vin.

First a  vin with no soh:
- nb_km / initial_wlpt


Second a vin with the soh:

- we have to take the degradation of the battery for each cycle

## Data load

In [ ]:
@cache_result('../tesla/data_cache/tesla_results.parquet', "local_storage")
def get_results() -> DF:
    return (
        TeslaProcessedTimeSeries("tesla", filters=[("trimmed_in_charge", "==", True)])
        .groupby(["vin", "trimmed_in_charge_idx"], observed=True, as_index=False)
        .agg(
            energy_added_min=pd.NamedAgg("charge_energy_added", "min"),
            energy_added_end=pd.NamedAgg("charge_energy_added", "last"),
            soc_diff=pd.NamedAgg("soc", series_start_end_diff),
            inside_temp=pd.NamedAgg("inside_temp", "mean"),
            capacity=pd.NamedAgg("capacity", "first"),
            odometer=pd.NamedAgg("odometer", "first"),
            version=pd.NamedAgg("version", "first"),
            size=pd.NamedAgg("soc", "size"),
            model=pd.NamedAgg("model", "first"),
            date=pd.NamedAgg("date", "first"),
            charging_power=pd.NamedAgg("charging_power", "median"),
            tesla_code=pd.NamedAgg("tesla_code", "first"),
            range=pd.NamedAgg("range", "first"),
            autonomy=pd.NamedAgg("autonomy", "first"),
        )
        .eval("energy_added = energy_added_end - energy_added_min")
        .eval("soh = energy_added / (soc_diff / 100.0 * capacity)")
        # .query("soc_diff > 40 & soh.between(0.75, 1.05)")
        .eval("level_1 = soc_diff * (charging_power < @LEVEL_1_MAX_POWER) / 100")
        .eval("level_2 = soc_diff * (charging_power.between(@LEVEL_1_MAX_POWER, @LEVEL_2_MAX_POWER)) / 100")
        .eval("level_3 = soc_diff * (charging_power > @LEVEL_2_MAX_POWER) / 100")
	    .eval("bottom_soh = soh.between(0.75, 0.9)")
        .eval("fixed_soh_min_end = soh.mask(tesla_code == 'MTY13', soh / 0.96)")
        .eval("fixed_soh_min_end = fixed_soh_min_end.mask(bottom_soh & tesla_code == 'MTY13', fixed_soh_min_end + 0.08)")
        .eval("soh = fixed_soh_min_end")
        .sort_values(["tesla_code", "vin", "date"])
    )

In [ ]:
from core.sql_utils import *
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT * FROM vehicle_data vd
            join vehicle v
            on v.id = vd.vehicle_id
            join vehicle_model vm 
            on vm.id = v.vehicle_model_id
            WHERE vm.model_name like '%model%';"""), con)


In [ ]:
df = get_results()

In [ ]:
# get the autonomy in the dataset
df = df.merge(dbeaver_df[['vin', 'autonomy']], on='vin')

## compute number of cycles

In [ ]:
# select a vin to try
df_vin = df[df['vin']=='LRWYGCFS7PC885103']

In [ ]:
autonomie_initiale =  df_vin['autonomy'].max()
soh = df_vin.soh.min() 
total_km = df_vin.odometer.max()  
print(autonomie_initiale, soh, total_km)

In [ ]:
# Sans considérer la dégradation par cycle
def estimate_cycles(total_range:float=0, initial_range:float=1, soh:float=1.0):
    """Calcule le nombre estimé de cycles

    Args:
        total_range (float): nombre de km parcouru
        initial_range (float): autonomie initiale du véhicule
        soh (float, optional): SoH du véhicule 

    Returns:
        float: le nombre de cycle de la batterie
    """
    if soh is np.nan:
        soh=1
    try:
        total_cycle = total_range / (initial_range * (soh + 1) / 2)
        return round(total_cycle)
    except:
        return np.nan


In [ ]:
estimate_cycles(total_km, autonomie_initiale, soh)